# Overview

#### Objective: Develop target list of companies to explore further. 
###### Traingulating:
- Growth
- Age
- Business health

### Config

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', 400)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
pd.options.display.float_format = '{:,}'.format
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# SBA Lending Data

### Loading Data

In [3]:
# lending_data_504 = pd.read_csv("data/FOIA - 504 (FY1991-Present).csv", encoding = "ISO-8859-1")

In [4]:
lending_data_7a_91t99 = pd.read_csv("data/FOIA - 7(a)(FY1991-FY1999).csv")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
lending_data_7a_00t09 = pd.read_csv("data/FOIA - 7(a)(FY2000-FY2009).csv", encoding="ISO-8859-1")

In [ ]:
lending_data_7a_10tPR = pd.read_csv("data/FOIA - 7(a)(FY2010-Present).csv")

In [ ]:
dict_7a = pd.read_csv("data/7a_FOIA_Data_Dictionary.csv")

In [ ]:
naics_codes = pd.read_csv("data/naics-codes.csv", header=None)

### Preprocess

In [ ]:
lending_data_7a_91t99.drop(['AsOfDate', 'Program'], axis=1, inplace=True)
lending_data_7a_00t09.drop(['AsOfDate', 'Program'], axis=1, inplace=True)
lending_data_7a_10tPR.drop(['AsOfDate', 'Program'], axis=1, inplace=True)

In [ ]:
dict_7a.columns = ['Field_Name', "Definition"]

In [ ]:
naics_codes.columns = ['code', 'desc']

In [ ]:
lending_data_7a_91t99 = pd.merge(lending_data_7a_91t99, naics_codes, left_on="NaicsCode", right_on='code')
lending_data_7a_00t09 = pd.merge(lending_data_7a_00t09, naics_codes, left_on="NaicsCode", right_on='code')
lending_data_7a_10tPR = pd.merge(lending_data_7a_10tPR, naics_codes, left_on="NaicsCode", right_on='code')

In [ ]:
lending_data_7a_91t99['group'] = '1991-1999'
lending_data_7a_00t09['group'] = '2000-2009'
lending_data_7a_10tPR['group'] = '2010-2020'

In [ ]:
lend_df = pd.concat([lending_data_7a_91t99,lending_data_7a_00t09,lending_data_7a_10tPR])

In [150]:
lend_df.to_csv("lending_data_7a_1991-2020.csv")

In [ ]:
lend_df

## Data Dictionary

In [1]:
dict_7a

NameError: name 'dict_7a' is not defined

### Experiment

# 1. Filtering companies using location, lending amts, headcount, etc.

In [ ]:
lend_df.reset_index()

# 2. How do interest rates, volume, etc. differ by groupings

In [151]:
lend_df.columns

Index(['BorrName', 'BorrStreet', 'BorrCity', 'BorrState', 'BorrZip',
       'BankName', 'BankStreet', 'BankCity', 'BankState', 'BankZip',
       'GrossApproval', 'SBAGuaranteedApproval', 'ApprovalDate',
       'ApprovalFiscalYear', 'FirstDisbursementDate', 'DeliveryMethod',
       'subpgmdesc', 'InitialInterestRate', 'TermInMonths', 'NaicsCode',
       'NaicsDescription', 'FranchiseCode', 'FranchiseName', 'ProjectCounty',
       'ProjectState', 'SBADistrictOffice', 'CongressionalDistrict',
       'BusinessType', 'LoanStatus', 'PaidInFullDate', 'ChargeOffDate',
       'GrossChargeOffAmount', 'RevolverStatus', 'JobsSupported', 'code',
       'desc', 'group'],
      dtype='object')

# 3. How to determine if company is still alive